# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [40]:
city_df = pd.read_csv('../WeatherPy/clean_weather.csv')
city_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date
0,Carthage,TN,36.8611,10.3316,68.00,45,20,11.50,1619274526
1,Mitsamiouli,KM,-11.3847,43.2844,82.40,83,75,11.50,1619274526
2,Mataura,NZ,-46.1927,168.8643,44.01,91,14,1.99,1619274526
3,Puerto Ayora,EC,-0.7393,-90.3518,77.00,88,75,5.75,1619274526
4,Atuona,PF,-9.8000,-139.0333,80.22,75,54,21.92,1619274527
...,...,...,...,...,...,...,...,...,...
558,Sinnamary,GF,5.3833,-52.9500,79.18,87,100,11.63,1619274621
559,Oranjemund,NaN,-28.5500,16.4333,64.33,74,10,9.98,1619274622
560,Meadow Lake,US,34.8014,-106.5436,47.26,45,1,9.01,1619274622
561,Stara Vyzhivka,UA,51.4376,24.4400,46.22,49,97,12.50,1619274622


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
#configure gmaps key
gmaps.configure(api_key=g_key)

# Setting the parameters for the map

humidity = city_df["Humidity"].astype(float)
maxhum = humidity.max()
locations = city_df[["Latitude", "Longitude"]]


In [42]:
# Generating the heat map.

fig = gmaps.figure()

# Describing the heat map
heat = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhum,point_radius=3)

fig.add_layer(heat)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
# Find cities ideal for vacation 
narrowed_city_df = city_df.loc[(city_df["Temperature"] > 70) & (city_df["Temperature"] < 80) & (city_df["Cloudiness"] == 0), :]

# Drop cities outside the ideal condition
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date
0,Arraial do Cabo,BR,-22.9661,-42.0278,78.80,69,0,11.50,1619274528
1,East London,ZA,-33.0153,27.9116,71.60,64,0,10.36,1619274529
2,Howard Springs,AU,-12.4970,131.0470,76.62,44,0,5.75,1619274553
3,Carnarvon,AU,-24.8667,113.6333,73.40,83,0,9.22,1619274294
4,Manavgat,TR,36.7867,31.4431,78.80,36,0,4.61,1619274562
5,Malatya Province,TR,38.5000,38.0000,78.80,15,0,4.61,1619274568
6,Kiryat Gat,IL,31.6100,34.7642,73.99,30,0,1.99,1619274598
7,Marsá Maţrūḩ,EG,31.3525,27.2453,77.00,23,0,11.50,1619274605
8,Asunción Mita,GT,14.3308,-89.7108,76.14,69,0,4.29,1619274613
9,Tataouine,TN,32.9297,10.4518,73.40,33,0,9.22,1619274621


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
# Empty list of hotel at ideal location
hotellist = []

# For loop to find the hotels at the ideal location

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Latitude']
    lng = narrowed_city_df.loc[i]['Longitude']

# setting the parameters
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()

# To find the first hotel from the ideal location
   

    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")

# Add column to the data frame with hotel names.

narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,Arraial do Cabo,BR,-22.9661,-42.0278,78.80,69,0,11.50,1619274528,Arraial do Cabo
1,East London,ZA,-33.0153,27.9116,71.60,64,0,10.36,1619274529,East London
2,Howard Springs,AU,-12.4970,131.0470,76.62,44,0,5.75,1619274553,Darwin
3,Carnarvon,AU,-24.8667,113.6333,73.40,83,0,9.22,1619274294,Carnarvon
4,Manavgat,TR,36.7867,31.4431,78.80,36,0,4.61,1619274562,Manavgat
5,Malatya Province,TR,38.5000,38.0000,78.80,15,0,4.61,1619274568,Ilıcak
6,Kiryat Gat,IL,31.6100,34.7642,73.99,30,0,1.99,1619274598,Kiryat Gat
7,Marsá Maţrūḩ,EG,31.3525,27.2453,77.00,23,0,11.50,1619274605,Marsa Matruh
8,Asunción Mita,GT,14.3308,-89.7108,76.14,69,0,4.29,1619274613,Asunción Mita
9,Tataouine,TN,32.9297,10.4518,73.40,33,0,9.22,1619274621,Tataouine


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitude", "Longitude"]]

In [47]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))